# Chapter 8: Reissner-Mindlin plate

Rectangular 2D linear plate.

In [14]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
sp.init_printing()

### The physical properties of the element

In [15]:
E = 210e9  # Young's modulus in Pa
nu = 0.3  # Poisson's ratio

# Define symbolic physical coordinates
x1, y1, x2, y2, x3, y3, x4, y4 = sp.symbols('x1 y1 x2 y2 x3 y3 x4 y4')
# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])

# Example values for the physical coordinates
x1_val, y1_val = -1, -1
x2_val, y2_val = 1, -1
x3_val, y3_val = 1, 1
x4_val, y4_val = -1, 1

x1_val, y1_val = -3, -2
x2_val, y2_val = 1.5, -0.7
x3_val, y3_val = 1, 1.2
x4_val, y4_val = -2, 1

# Create a dictionary to map the symbolic coordinates to their numerical values
coordinate_values = {
    x1: x1_val, y1: y1_val,
    x2: x2_val, y2: y2_val,
    x3: x3_val, y3: y3_val,
    x4: x4_val, y4: y4_val,
}

### The finite element

This is a rectangular isoparametric 2D plate in the x-y plane.

The natural coordinates are xi and eta. Node numbering is CCW, starting at ``(-1, -1)``:
```
     ^
     |eta
     |
  4--|--3
  |  |--|--->xi
  |     |
  1-----2
```

It has 3 degrees of freedom each node:
- w: vertical displacement
- thetaX: rotation about the x-axis
- thetaY: rotation about the y-axis
The DOFs are not coupled.
The DOFs are ordered as follows: w, thetaX, thetaY.


In [16]:
ND = 3  # number of degrees of freedom per node
NNODE = 4  # number of nodes

### The isoparametric part

In [17]:
n1 = np.array([-1, -1])  # node 1 coordinates
n2 = np.array([1, -1])   # node 2 coordinates
n3 = np.array([1, 1])    # node 3 coordinates
n4 = np.array([-1, 1])   # node 4 coordinates
natural_nodes = np.array([n1, n2, n3, n4])  # all nodes

### Shape functions and their derivatives

In [27]:
# Define symbolic variables
XI, ETA = sp.symbols('xi eta')

# Define the shape functions symbolically
def shape_functions() -> np.array:
    """shape functions"""

    return np.array([
        0.25 * (1 - XI) * (1 - ETA),
        0.25 * (1 + XI) * (1 - ETA),
        0.25 * (1 + XI) * (1 + ETA),
        0.25 * (1 - XI) * (1 + ETA),
    ])

def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    print(shape_functions())

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 1])])


# Calculate the derivatives by xi and eta, symbolically
def shape_function_derivatives() -> np.array:
    """shape function derivatives"""
    sh = shape_functions()
    dN_dxi = []
    dN_deta = []

    for i in range(len(sh)):
        dN_dxi.append(sp.diff(sh[i], XI))
        dN_deta.append(sp.diff(sh[i], ETA))

    return dN_dxi, dN_deta

print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i+1}/dxi = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i+1}/deta = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

print()
print('The derivatives of the shape functions by xi are:')
print(derivatives[0])  # derivatives by xi

print()
print('The derivatives of the shape functions by eta are:')
print(derivatives[1])  # derivatives by eta


The shape functions are:
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]

The derivatives are:
dN1/dxi = 0.25*eta - 0.25
dN2/dxi = 0.25 - 0.25*eta
dN3/dxi = 0.25*eta + 0.25
dN4/dxi = -0.25*eta - 0.25
dN1/deta = 0.25*xi - 0.25
dN2/deta = -0.25*xi - 0.25
dN3/deta = 0.25*xi + 0.25
dN4/deta = 0.25 - 0.25*xi

The isoparametric mapping is:
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]
[(0.25 - 0.25*xi)*(1 - eta) (1 - eta)*(0.25*xi + 0.25)
 (eta + 1)*(0.25*xi + 0.25) (0.25 - 0.25*xi)*(eta + 1)]
x = x1*(0.25 - 0.25*xi)*(1 - eta) + x2*(1 - eta)*(0.25*xi + 0.25) + x3*(eta + 1)*(0.25*xi + 0.25) + x4*(0.25 - 0.25*xi)*(eta + 1)
y = y1*(0.25 - 0.25*xi)*(1 - eta) + y2*(1 - eta)*(0.25*xi + 0.25) + y3*(eta + 1)*(0.25*xi + 0.25) + y4*(0.25 - 0.25*xi)*(eta + 1)

The derivatives of the shape functions by xi are:
[0.25*eta - 0.25, 0.25 - 0.25*eta, 0.25*eta + 0.25, -0.25*eta - 0.2

### Example of the shape functions

In [29]:
# Evaluate the shape functions at xi=0, eta=0
shape_funcs_at = shape_functions()
print("Shape functions at xi=0, eta=0:")

for i, sf in enumerate(shape_funcs_at):
    print(f"N{i+1} = {sf.subs({XI: 0, ETA: 0})}")

print()
shape_funcs_at = shape_functions()
print("Shape functions at xi=1, eta=1:")
for i, sf in enumerate(shape_funcs_at):
    print(f"N{i+1} = {sf.subs({XI: 1, ETA: 1})}")


Shape functions at xi=0, eta=0:
N1 = 0.250000000000000
N2 = 0.250000000000000
N3 = 0.250000000000000
N4 = 0.250000000000000

Shape functions at xi=1, eta=1:
N1 = 0
N2 = 0
N3 = 1.00000000000000
N4 = 0


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4 & 0 & 0 \\
0 & N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4 & 0\\
0 & 0 & N_1 & 0 & 0 & N_2 & 0 & 0 & N_3 & 0 & 0 & N_4
\end{bmatrix}
$$

In [40]:
def N() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[N1, 0, 0, N2, 0, 0, N3, 0, 0, N4, 0, 0],
     [0, N1, 0, 0, N2, 0, 0, N3, 0, 0, N4],
     [0, 0, N1, 0, 0, N2, 0, 0, N3, 0, 0]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

print("The shape function matrix N is:")
sp.pprint(N())

The shape function matrix N is:
⎡(0.25 - 0.25⋅ξ)⋅(1 - η)             0                        0             (1 ↪
⎢                                                                              ↪
⎢           0             (0.25 - 0.25⋅ξ)⋅(1 - η)             0                ↪
⎢                                                                              ↪
⎣           0                        0             (0.25 - 0.25⋅ξ)⋅(1 - η)     ↪

↪  - η)⋅(0.25⋅ξ + 0.25)             0                        0             (η  ↪
↪                                                                              ↪
↪          0             (1 - η)⋅(0.25⋅ξ + 0.25)             0                 ↪
↪                                                                              ↪
↪          0                        0             (1 - η)⋅(0.25⋅ξ + 0.25)      ↪

↪ + 1)⋅(0.25⋅ξ + 0.25)             0                        0             (0.2 ↪
↪                                                                          

### The shape function matrix with values substituted

In [39]:
# Evaluate the shape function matrix at xi=0.5, eta=1.0
shape_function_matrix = N().subs({XI: 0.5, ETA: 1.0})
print("Shape function matrix N at xi=0.5, eta=1.0:")
sp.pprint(shape_function_matrix)

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Shape function matrix N at xi=0.5, eta=1.0:
⎡0  0  0  0  0  0  0.75   0     0    0.25   0     0  ⎤
⎢                                                    ⎥
⎢0  0  0  0  0  0   0    0.75   0     0    0.25   0  ⎥
⎢                                                    ⎥
⎣0  0  0  0  0  0   0     0    0.75   0     0    0.25⎦


### The jacobian matrix

The jacobian matrix does the following:
- maps the natural coordinates (xi, eta) to the physical coordinates (x, y) as it a measure how the physical coordinates change when the natural coordinates change.
- it is used to transform the derivatives of the shape functions from the natural coordinates to the physical coordinates using the chain rule.

So the jacobian is constructed using the chain rule, from the derivatives of the mapping between the global and the local system, and its value depends on the physical coordinates. The Jacobian changes within the element because the mapping from the natural coordinates (xi, eta) to the physical coordinates (x, y) is not necessarily linear. The derivatives of the mapping functions (x_map, y_map) with respect to xi and eta (which form the Jacobian) depend on the shape and distortion of the element in physical space. Since xi and eta vary across the element, the derivatives, and thus the Jacobian, also vary.

$$J = \begin{bmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial y}{\partial \xi} \\
\frac{\partial x}{\partial \eta} & \frac{\partial y}{\partial \eta}
\end{bmatrix}$$

The chain rule for derivation is:

$$\begin{bmatrix}
\frac{\partial}{\partial \xi} \\
\frac{\partial}{\partial \eta}
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial y}{\partial \xi} \\
\frac{\partial x}{\partial \eta} & \frac{\partial y}{\partial \eta}
\end{bmatrix}
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}
= J
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}$$

To express the derivatives with respect to $x$ and $y$ in terms of $\xi$ and $\eta$, you would use the inverse of the Jacobian:

$$
\begin{bmatrix}
\frac{\partial}{\partial x} \\
\frac{\partial}{\partial y}
\end{bmatrix}
= J^{-1}
\begin{bmatrix}
\frac{\partial}{\partial \xi} \\
\frac{\partial}{\partial \eta}
\end{bmatrix}
$$

$$J^{-1} = \begin{bmatrix}
\frac{\partial \xi}{\partial x} & \frac{\partial \xi}{\partial y} \\
\frac{\partial \eta}{\partial x} & \frac{\partial \eta}{\partial y}
\end{bmatrix}$$

The determinant of the jacobian is used to account for the change of the element area when transforming from the natural to the physical coordinates.

In [9]:
def jacobian() -> np.array:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # Calculate the derivatives
    dx_dxi = sp.diff(x_map, XI)
    dy_dxi = sp.diff(y_map, XI)
    dx_deta = sp.diff(x_map, ETA)
    dy_deta = sp.diff(y_map, ETA)

    return sp.Matrix([[dx_dxi, dy_dxi],
                      [dx_deta, dy_deta]])

# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(coordinate_values)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at all nodes
for xi_eval in [-1, 0, 1]:
    for eta_eval in [-1, 0, 1]:
        J_evaluated = J_substituted.subs({XI: xi_eval, ETA: eta_eval})
        print(f'Jacobian matrix at xi={xi_eval}, eta={eta_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2f}')
        print()


The jacobian matrix with coordinate values substituted is:
⎡1.875 - 0.375⋅η  0.375 - 0.275⋅η⎤
⎢                                ⎥
⎣0.125 - 0.375⋅ξ  1.225 - 0.275⋅ξ⎦

Jacobian matrix at xi=-1, eta=-1:
⎡2.25  0.65⎤
⎢          ⎥
⎣0.5   1.5 ⎦
The determinant at this point is |J|=3.05

Jacobian matrix at xi=-1, eta=0:
⎡1.875  0.375⎤
⎢            ⎥
⎣ 0.5    1.5 ⎦
The determinant at this point is |J|=2.62

Jacobian matrix at xi=-1, eta=1:
⎡1.5  0.1⎤
⎢        ⎥
⎣0.5  1.5⎦
The determinant at this point is |J|=2.20

Jacobian matrix at xi=0, eta=-1:
⎡2.25   0.65 ⎤
⎢            ⎥
⎣0.125  1.225⎦
The determinant at this point is |J|=2.68

Jacobian matrix at xi=0, eta=0:
⎡1.875  0.375⎤
⎢            ⎥
⎣0.125  1.225⎦
The determinant at this point is |J|=2.25

Jacobian matrix at xi=0, eta=1:
⎡ 1.5    0.1 ⎤
⎢            ⎥
⎣0.125  1.225⎦
The determinant at this point is |J|=1.82

Jacobian matrix at xi=1, eta=-1:
⎡2.25   0.65⎤
⎢           ⎥
⎣-0.25  0.95⎦
The determinant at this point is |J|=2.30

Jacobian m

### Visualizing the determinant of the jacobian

In [11]:
%matplotlib notebook

# Calculate the determinant of the Jacobian
J_det = J.det()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_det_substituted = J_det.subs(coordinate_values)

# Create a function to evaluate the determinant for given xi and eta
J_det_func = sp.lambdify((XI, ETA), J_det_substituted)

# Create a grid of xi and eta values
xi_vals = np.linspace(-1, 1, 50)
eta_vals = np.linspace(-1, 1, 50)
xi_grid, eta_grid = np.meshgrid(xi_vals, eta_vals)

# Evaluate the determinant on the grid
det_values = J_det_func(xi_grid, eta_grid)

# Map xi and eta values to x and y coordinates
x_vals = sp.lambdify((XI, ETA), x_map.subs(coordinate_values))(xi_grid, eta_grid)
y_vals = sp.lambdify((XI, ETA), y_map.subs(coordinate_values))(xi_grid, eta_grid)

# move the physical element so the centroid is at 0, 0
x_vals -= np.mean([x1_val, x2_val, x3_val, x4_val])
y_vals -= np.mean([y1_val, y2_val, y3_val, y4_val])

# Create the contour plot
plt.figure()
contour = plt.contourf(x_vals, y_vals, det_values, levels=50, cmap='viridis')

# plot the natural element shape, too
plt.plot([-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1], 'k--', label='Natural Element Shape')

plt.colorbar(contour, label='det(J)')
plt.xlabel('xi')
plt.ylabel('eta')
plt.axis('equal')
plt.show()

<IPython.core.display.Javascript object>

### Deriving the $B^I$ matrix
This matrix iss associated with the in-plane strains.
$$B^I = \begin{bmatrix}B^I_1 & B^I_2 & B^I_3 & B^I_4\end{bmatrix}$$

where $B^I_j$ is the matrix for node $j$:
$$B^I_j = \begin{bmatrix}
0 & 0 & -\frac{\partial N_j}{\partial x} \\
0 & \frac{\partial N_j}{\partial y} & 0 \\
0 & \frac{\partial N_j}{\partial x} & -\frac{\partial N_j}{\partial y} \\
\end{bmatrix}$$

and

$$\frac{\partial N_j}{\partial x} = \frac{\partial N_j}{\partial \xi} \frac{\partial \xi}{\partial x}$$

wich uses the inverse jacobian matrix to transform the derivatives from the natural coordinates to the physical coordinates.

In [43]:
def BI(xi: float, eta: float) -> np.array:
    """
    Calculate the BI matrix for the element.
    """
    # the derivatives by the natural coordinates
    dN_dxi, dN_deta = shape_function_derivatives(xi, eta)

    print(f'dN_dxi: {type(dN_dxi)}')
    print(f'dN_deta: {dN_deta}')
    print(dN_dxi[0])
    print(dN_dxi[0].subs({XI: xi, ETA: eta}))

    # the inverse jacobian that contains the terms to transform the derivatives. This has 2x2 elements.
    # In row 1 are dxi/dx, dxi/dy
    # In row 2 are deta/dx, deta/dy
    Jinv = J_substituted.inv()  # The inverse of the jacobian

    B_I = None  # Initialize B_I to None

    for node in range(NNODE):
        # Calculate the derivatives by x and y
        dN_dx = dN_dxi[node] * Jinv[0, 0] + dN_deta[node] * Jinv[1, 0]
        dN_dy = dN_dxi[node] * Jinv[0, 1] + dN_deta[node] * Jinv[1, 1]

        # Create the B^I matrix for this node
        B_I_node = sp.Matrix([[0, 0, -dN_dx],
                             [0, dN_dy, 0],
                             [dN_dx, -dN_dy, 0]])

        if B_I is None:
            B_I = B_I_node
        else:
            B_I = sp.Matrix.hstack(B_I, B_I_node)

    return B_I

sp.pprint(BI(0.5, 0))

dN_dxi: <class 'list'>
dN_deta: [0, 0, 0, 0]
0
0
⎡0  0  0  0  0  0  0  0  0  0  0  0⎤
⎢                                  ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                  ⎥
⎣0  0  0  0  0  0  0  0  0  0  0  0⎦


### Deriving the $B^O$ matrix
This matrix iss associated with the off-plane strains.
$$B^I = \begin{bmatrix}B^O_1 & B^O_2 & B^O_3 & B^O_4\end{bmatrix}$$

where $B^O_j$ is the matrix for node $j$:
$$B^I_j = \begin{bmatrix}
\frac{\partial N_j}{\partial x} & 0 & N_j \\
\frac{\partial N_j}{\partial y} & -N_j & 0 \\
\end{bmatrix}$$

In [23]:
def BO(xi: float, eta: float) -> np.array:
    """
    Calculate the BO matrix for the element.
    """
    # the shape functions
    N = shape_functions(xi, eta)

    # the derivatives by the natural coordinates
    dN_dxi, dN_deta = shape_function_derivatives(xi, eta)

    # the inverse jacobian that contains the terms to transform the derivatives. This has 2x2 elements.
    # In row 1 are dxi/dx, dxi/dy
    # In row 2 are deta/dx, deta/dy
    Jinv = J_substituted.inv()  # The inverse of the jacobian

    B_O = None  # Initialize B_O to None

    for node in range(NNODE):
        # Calculate the derivatives by x and y
        dN_dx = dN_dxi[node] * Jinv[0, 0] + dN_deta[node] * Jinv[1, 0]
        dN_dy = dN_dxi[node] * Jinv[0, 1] + dN_deta[node] * Jinv[1, 1]

        # Create the B^O matrix for this node
        B_O_node = sp.Matrix([[dN_dx, 0, N[node]],
                              [dN_dy, -N[node], 0]])

        if B_O is None:
            B_O = B_O_node
        else:
            B_O = sp.Matrix.hstack(B_O, B_O_node)

    return B_O

sp.pprint(BO(0.5, 0))

Matrix([[(0.25*eta - 0.25)*(-0.25*xi*y1 + 0.25*xi*y2 - 0.25*xi*y3 + 0.25*xi*y4 + 0.25*y1 + 0.25*y2 - 0.25*y3 - 0.25*y4)/(0.125*eta*x1*y2 - 0.125*eta*x1*y3 - 0.125*eta*x2*y1 + 0.125*eta*x2*y4 + 0.125*eta*x3*y1 - 0.125*eta*x3*y4 - 0.125*eta*x4*y2 + 0.125*eta*x4*y3 + 0.125*x1*xi*y3 - 0.125*x1*xi*y4 - 0.125*x1*y2 + 0.125*x1*y4 - 0.125*x2*xi*y3 + 0.125*x2*xi*y4 + 0.125*x2*y1 - 0.125*x2*y3 - 0.125*x3*xi*y1 + 0.125*x3*xi*y2 + 0.125*x3*y2 - 0.125*x3*y4 + 0.125*x4*xi*y1 - 0.125*x4*xi*y2 - 0.125*x4*y1 + 0.125*x4*y3) + (0.25*xi - 0.25)*(0.25*x1*xi - 0.25*x1 - 0.25*x2*xi - 0.25*x2 + 0.25*x3*xi + 0.25*x3 - 0.25*x4*xi + 0.25*x4)/(0.125*eta*x1*y2 - 0.125*eta*x1*y3 - 0.125*eta*x2*y1 + 0.125*eta*x2*y4 + 0.125*eta*x3*y1 - 0.125*eta*x3*y4 - 0.125*eta*x4*y2 + 0.125*eta*x4*y3 + 0.125*x1*xi*y3 - 0.125*x1*xi*y4 - 0.125*x1*y2 + 0.125*x1*y4 - 0.125*x2*xi*y3 + 0.125*x2*xi*y4 + 0.125*x2*y1 - 0.125*x2*y3 - 0.125*x3*xi*y1 + 0.125*x3*xi*y2 + 0.125*x3*y2 - 0.125*x3*y4 + 0.125*x4*xi*y1 - 0.125*x4*xi*y2 - 0.125*x4*y1 

### Let's see a real world example
The plate is simply the same as the one in the natural coordinates

In [24]:


# Evaluate BI and BO at xi=0, eta=0
bi_matrix = BI(0, 0)
bo_matrix = BO(0, 0)

# Substitute the symbolic coordinates with numerical values
bi_matrix_evaluated = bi_matrix.subs(coordinate_values)
bo_matrix_evaluated = bo_matrix.subs(coordinate_values)

print("BI Matrix at xi=0, eta=0 with substituted coordinates:")
sp.pprint(bi_matrix_evaluated)
print()
print("BO Matrix at xi=0, eta=0 with substituted coordinates:")
sp.pprint(bo_matrix_evaluated)

ValueError: First variable cannot be a number: 0